In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

D:\Anaconda documentation\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### Pre processing img data 

In [2]:
train_dir = "D:\PES MCOE\semester 7\DL\DL LP 4 Datasets\cifar-10-img/train"
test_dir = "D:\PES MCOE\semester 7\DL\DL LP 4 Datasets\cifar-10-img/test"

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)

# here batch_size is the number of images in each batch
train_batch_size = 5000
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=train_batch_size,
    class_mode='categorical'
)
test_batch_size = 1000
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(32, 32),
    batch_size=test_batch_size,
    class_mode='categorical'
)

Found 40079 images belonging to 10 classes.
Found 9921 images belonging to 10 classes.


#### Selecting only first batch with 5000 images as train and test data

In [3]:
x_train, y_train =  train_generator[0]
x_test, y_test = test_generator[0]

print(len(x_train))
print(len(x_test))

5000
1000


#### a. Load in a pre-trained CNN model trained on a large dataset

In [5]:
# Load VGG16 without top layers
weights_path = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

#### b. Freeze parameters (weights) in model’s lower convolutional layers

In [6]:
for layer in base_model.layers:
    layer.trainable = False

#### c. Add custom classifier with several layers of trainable parameters to model

In [7]:
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
predictions = Dense(10, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)
# Compile the model
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

#### d. Train classifier layers on training data available for task

In [8]:
# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10


79/79 [==============================] - 19s 223ms/step - loss: 1.9762 - accuracy: 0.2792 - val_loss: 1.6297 - val_accuracy: 0.4350
Epoch 2/10
79/79 [==============================] - 21s 269ms/step - loss: 1.5907 - accuracy: 0.4412 - val_loss: 1.4623 - val_accuracy: 0.4850
Epoch 3/10
79/79 [==============================] - 21s 270ms/step - loss: 1.4284 - accuracy: 0.4958 - val_loss: 1.4296 - val_accuracy: 0.5020
Epoch 4/10
79/79 [==============================] - 22s 273ms/step - loss: 1.3430 - accuracy: 0.5324 - val_loss: 1.3871 - val_accuracy: 0.5210
Epoch 5/10
79/79 [==============================] - 23s 286ms/step - loss: 1.2725 - accuracy: 0.5470 - val_loss: 1.3934 - val_accuracy: 0.5080
Epoch 6/10
79/79 [==============================] - 25s 314ms/step - loss: 1.2334 - accuracy: 0.5734 - val_loss: 1.3348 - val_accuracy: 0.5430
Epoch 7/10
79/79 [==============================] - 23s 296ms/step - loss: 1.1589 - accuracy: 0.5934 - val_loss: 1.3389 - val_accuracy: 0.54

#### e. Fine-tune hyper parameters and unfreeze more layers as needed

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# freeze all layers first
for layer in base_model.layers:
    layer.trainable = False

# unfreeze last 4 layers of base model
for layer in base_model.layers[len(base_model.layers) - 4:]:
    layer.trainable = True

# fine-tuning hyper parameters
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
predictions = Dense(10, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# training fine tuned model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
79/79 [==============================] - 28s 339ms/step - loss: 2.2769 - accuracy: 0.1210 - val_loss: 2.3041 - val_accuracy: 0.0990
Epoch 2/10
79/79 [==============================] - 31s 388ms/step - loss: 2.3036 - accuracy: 0.1022 - val_loss: 2.3015 - val_accuracy: 0.1130
Epoch 3/10
79/79 [==============================] - 30s 377ms/step - loss: 2.3028 - accuracy: 0.1028 - val_loss: 2.3018 - val_accuracy: 0.1130
Epoch 4/10
79/79 [==============================] - 30s 382ms/step - loss: 2.3029 - accuracy: 0.1060 - val_loss: 2.3018 - val_accuracy: 0.1130
Epoch 5/10
79/79 [==============================] - 31s 390ms/step - loss: 2.3031 - accuracy: 0.1046 - val_loss: 2.3019 - val_accuracy: 0.1130
Epoch 6/10
79/79 [==============================] - 30s 385ms/step - loss: 2.3027 - accuracy: 0.1050 - val_loss: 2.3020 - val_accuracy: 0.1130
Epoch 7/10
79/79 [==============================] - 32s 406ms/step - loss: 2.3026 - accuracy: 0.1062 - val_loss: 2.3017 - val_accuracy: 0.1130

In [ ]:
import matplotlib.pyplot as plt
predicted_value = model.predict(x_test)

In [ ]:
labels = list(test_generator.class_indices.keys())

In [ ]:
n = 890
plt.imshow(x_test[n])
print("Preditcted: ",labels[np.argmax(predicted_value[n])])
print("Actual: ", labels[np.argmax(y_test[n])])